In [1]:
import sys
import numpy as np

from vispy import app, scene, color
from vispy.util.filter import gaussian_filter

import gdal

In [2]:
# read terrain
# DTM_Inca_City_ngate_1m_forPDS_no_special.cub
# 12767 6985
# ESP_022607_0985_REDmos_hijitreged_1m_o_forPDS.cub
# 12767 6985
# ESP_022607_0985_REDmos_hijitreged_25cm_o_forPDS.cub
# 51068 27940
# ESP_022607_0985_cropped_big_spider.cub
# 397 455
# ESP_022699_0985_REDmos_hijitreged_1m_o_forPDS.cub
# 12767 6985
# ESP_022699_0985_REDmos_hijitreged_25cm_o_forPDS.cub
# 51068 27940
# ESP_022699_0985_cropped_big_spider.cub
# 397 455
# SS_DTM_Inca_City_ngate_1m.cub
# 12767 6985
# ds = gdal.Open('/Volumes/Data/hirise/dem/inca/big_spider_dem.cub')
from pathlib import Path
root = Path('/Users/klay6683/data/hirise/dem/inca')
fulldata = {'dem': root / 'DTM_Inca_City_ngate_1m_forPDS_no_special.cub',
       'data1': root / 'ESP_022607_0985_REDmos_hijitreged_1m_o_forPDS.cub',
       'data2': root / 'ESP_022699_0985_REDmos_hijitreged_1m_o_forPDS.cub'}
spider = {'dem': root / 'big_spider_dem.cub',
          'data1': root / 'ESP_022607_0985_cropped_big_spider.cub',
          'data2': root / 'ESP_022699_0985_cropped_big_spider.cub'}

In [3]:
cmap = color.get_colormap('cubehelix')
from skimage.exposure import rescale_intensity
from skimage.color import gray2rgb
def load_data(datadic, xsize=500, ysize=500):
    retdata = {}
    for k,v in datadic.items():
        ds = gdal.Open(str(v))
        if ds is None:
            raise FileNotFoundError
        actx = ds.RasterXSize
        acty = ds.RasterYSize
        if xsize > actx or ysize > acty:
            retdata[k] = ds.ReadAsArray()
        else:
            retdata[k] = ds.ReadAsArray(xoff=0, yoff=0,
                                        xsize=xsize,
                                        ysize=ysize)
    # shifting the DEM down because camera movement does not
    # work properly
    retdata['dem'] = retdata['dem'] - retdata['dem'].min()
    retdata['data1'] = rescale_intensity(retdata['data1'].astype('float'))
    retdata['data2'] = rescale_intensity(retdata['data2'].astype('float'))
    return retdata

In [4]:
container = load_data(spider)

In [5]:
canvas = scene.SceneCanvas(keys='interactive', size=(1024, 768))
view = canvas.central_widget.add_view()
view.camera = scene.TurntableCamera(up='z')

In [6]:
shape = container['data1'].shape
colors = cmap.map(container['data1'])

In [7]:
colors.shape

(180635, 4)

In [11]:
colors = colors.reshape(shape + (4,))

In [11]:
colors.max()

1.0

In [6]:
#view.camera.center = [data.shape[0]/2, data.shape[1]/2, 0]

In [53]:
scene.visuals.SurfacePlot?

In [9]:
container['dem'].shape

(455, 397)

In [12]:
colors.shape

(455, 397, 4)

In [13]:
p1 = scene.visuals.SurfacePlot(z=container['dem'],
                               colors=colors,
                               shading='smooth')

ValueError: colors must be 2D if indexed is None

In [32]:
p1.transform = scene.transforms.AffineTransform()

In [33]:
p1.transform.scale([1., 1., 2.])
# p1.transform.translate([-0.5, -0.5, 0])

In [34]:
view.add(p1)

In [35]:
axis=scene.visuals.XYZAxis(parent=view.scene)

In [36]:
p1.transform.translate([0.,])

In [37]:
view.camera.set_range()

In [38]:
canvas.show()
app.run()

0

In [39]:
data.shape

(500, 500)

In [40]:
data.dtype

dtype('float32')